In [ ]:
import os
import cv2
import numpy as np

def calculate_covariances(frame, window_size):
    h, w, _ = frame.shape
    max_cov = -np.inf
    max_window = None
    for i in range(0, h - window_size[1] + 1):
        for j in range(0, w - window_size[0] + 1):
            window = frame[i:i + window_size[1], j:j + window_size[0], :]
            red_channel = window[:, :, 2].flatten()  # Red intensities
            green_channel = window[:, :, 1].flatten()  # Green intensities
            blue_channel = window[:, :, 0].flatten()  # Blue intensities
            
            cov_rg = np.cov(red_channel, green_channel)[0, 1]
            cov_rb = np.cov(red_channel, blue_channel)[0, 1]
            
            if abs(cov_rg) + abs(cov_rb) > max_cov:
                max_cov = abs(cov_rg) + abs(cov_rb)
                max_window = (j, i, window_size[0], window_size[1])
    return max_window

def process_video(input_path, output_path, window_size):
    cap = cv2.VideoCapture(input_path)
    
    # original width, height, and fps
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    ret, first_frame = cap.read()
    if not ret:
        print(f"Failed to read the first frame of {input_path}")
        return
    
    # coordinates for the window with max covariance in the first frame
    max_window = calculate_covariances(first_frame, window_size)
    if max_window is None:
        print(f"No valid window found for {input_path}")
        return
    
    x, y, crop_width, crop_height = max_window
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (crop_width, crop_height))
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            cropped_frame = frame[y:y + crop_height, x:x + crop_width]
            
            out.write(cropped_frame)
        else:
            break
    
    cap.release()
    out.release()

input_dir = 'sample_videos'
output_dir = 'cropped_videos'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

video_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

for video_file in video_files:
    input_video_path = os.path.join(input_dir, video_file)
    output_video_path = os.path.join(output_dir, 'cropped_' + video_file)
    window_size = (11, 11)
    
    process_video(input_video_path, output_video_path, window_size)

print("Cropping completed.")

Cropping completed.
